<div class="alert alert-block alert-info">
<b>Note:</b> Before trying to scrap any website you should inspect that webpage whether the content that you want is present the HTML tags or not.</div>

## Step 1: Importing the Required Libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
print("Import Successful")

Import Successful


## Step 2: Specifying the URL

In [2]:
url="https://www.flipkart.com/audio-video/speakers/pr?sid=0pm%2C0o7&q=speaker&otracker=categorytree&sort=relevance&page=1"
url
#Speakers on flipkart

'https://www.flipkart.com/audio-video/speakers/pr?sid=0pm%2C0o7&q=speaker&otracker=categorytree&sort=relevance&page=1'

We also have edited the website by trimming the last part of the url which was for the page no. 1 

## Step 3: Sending a Request and Checking the Response

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; The **`requests.get(url)`** function sends a GET request to the specified URL using the HTTP protocol. The server responds with an HTTP response object, which contains information about the response, such as the status code, headers, and content.

In [3]:
r=requests.get(url)
print(r)

<Response [200]>


<div class="alert alert-block alert-success">
<b>Success:</b> Here, Response 200 shows that we are good to go. 
</div>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;A response status code of **503** indicates that the server is currently unable to handle the request due to a temporary overload or maintenance of the server. **404**,**400** are some errors faced while scraping. So it is better to check the response status of the website first before proceeding further.

## Step 4: Parsing the Response Content

In [4]:
soup=BeautifulSoup(r.text,"lxml")
# print(soup.prettify())

## Step 5: Extracting the Desired Data

In [8]:
# Create a dictionary to store lists with keys and values
data_dict = {}

In [5]:
def extract_table_data(table_element):
    data = {}
    rows = table_element.find_all('tr', class_='_1s_Smc')
    for row in rows:
        columns = row.find_all(['td', 'th'])
        if len(columns) == 2:
            key = columns[0].text.strip()
            value = columns[1].text.strip()
            data[key] = value
    return data

In [6]:
def looping_on_page():
    for link in links:   
        url=link
        r=requests.get(url)
        soup=BeautifulSoup(r.text,"lxml")
        
        # Find all tables with class '_14cfVK'
        tables = soup.find_all('table', class_='_14cfVK')
        
        # Extract and store data from each table in the dictionary
        for table in tables:
            table_data = extract_table_data(table)
            for key, value in table_data.items():
                if key not in data_dict:
                    data_dict[key] = []
                data_dict[key].append(value)

In [7]:
def speaker_scrape():
    looping_on_page()
    # Find the maximum length among all lists
    max_length = max(len(lst) for lst in data_dict.values())
    
    # Fill shorter lists with None to make them of equal length
    for key in data_dict:
        data_dict[key] += [None] * (max_length - len(data_dict[key]))
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data_dict)
    
    return df

## Step 6: Scraping Multiple Pages (Optional)

In [9]:
total_pages=506
for page in range(total_pages):
    progress = (page + 1) / total_pages * 100
    print(f"Progress: {progress: .2f}%",f"Page No.:{page+1}",end="\r")
    url="https://www.flipkart.com/audio-video/speakers/pr?sid=0pm%2C0o7&q=speaker&otracker=categorytree&sort=relevance&page={}".format(page)
    request=requests.get(url).text
    soup=BeautifulSoup(request,"lxml")
    links = [f"https://www.flipkart.com{a['href']}" for a in soup.find_all('a', class_='_2rpwqI')]
    df1 = speaker_scrape()

In [19]:
data_df = pd.DataFrame(data_dict)
data_df.head()

,Sales Package,Model Number,Model Name,Type,Portable,Bluetooth,Memory Card Slot,Configuration,Power Source,Frequency Response,...,Subwoofer Depth,Tweeter Diameter,International Warranty,Satellite Width,Satellite Height,Satellite Depth,Center Satellite Width,Center Satellite Height,Center Satellite Depth,Center Satellite Weight
0,1 x 2.0 USB Speaker,Mini 2.0 USB Multimedia Speakers,Mini,Laptop/Desktop Speaker,Yes,No,No,2.0,USB,60Hz-20Khz,...,313 mm,16 mm,1 Year,105 mm,173 mm,83 mm,87.5 mm,147 mm,87.5 mm,1.2 kg
1,"2 wired speakers with wired volume controller,...",S21,S21,Laptop/Desktop Speaker,Yes,No,No,2.0 Channel,DC 5V via a USB port,100hz-20khz,...,222 mm,65 mm,1 Year,9 mm,17.2 mm,10.2 mm,None,None,None,None
2,"1 N Speaker, 1 N Charging Cable, 1 N AUX Cable",M423SP (PORTABLE HOME TV SOUNDBAR) Dynamic Thu...,Powerpact bass & stereo audio color changIng l...,Soundbar,Yes,Yes,Yes,2.0,"Rechargeable, USB",150hz—20khz,...,None,20 mm,None,115 mm,95 mm,320 mm,None,None,None,None
3,"1 x Bluetooth speaker, 1 x USB charging cable",Powerpact bass & stereo audio color changIng l...,"Fort S100 with 2 in-built subwoofers, Made in ...",Mobile/Tablet Speaker,Yes,3,Yes,2.0 Channel,Battery(Charge At Least 1 hour Before 1st Use)...,150 Hz- 20 KHz,...,None,None,None,None,None,None,None,None,None,None
4,"Sales Package:�Soundbar, AUX in Cable, User Ma...",Aavante Bar 500 / 503 / 508,"Fort S16 Soundbar with 2 full range drivers, M...",Soundbar,None,Yes,No,Stereo,DC Adapter,80 - 20000 Hz,...,None,None,None,None,None,None,None,None,None,None


## Step 7: Saving the Scraped Data

In [26]:
data_df.to_csv("CSV Files/Speakers.csv") #Converts pandas dataframe into a csv file